___
# Atividade: Regressão Linear Simples
___

## Aula 21

**Preparo Prévio:**
1. Montogmery. Seção 6-2 - Simple Linear Regression
1. Magalhães e Lima, seção 9.5. Regressão Linear Simples

**Hoje:**
1. Entender o conceito de regressão linear

**Próxima aula: (Terça-feira)**
1. Projeto 3

**Próxima aula: (Quinta-feira)**
1. Montogmery. Seção 6-3 - Multiple Regression

___

##  Renda per capita vs CO2 de países

Foram coletados dois dados do site https://www.gapminder.org/:
1. Emissão de CO2 per capita
1. Renda per capita (sendo usado PBI como uma proxy de renda)

Nesses *dataframes*, as linhas representam os países, as colunas representam o ano.

No *dataframe* *co2* criado a seguir, o conteúdo de cada célula é a medida de CO2 de um determinado ano (coluna) para determinado país (linha). 

No *dataframe* *income* criado a seguir, o conteúdo de cada célula é a medida de PIB per capita de um determinado ano (coluna) para um determinado país (linha). 

Como os arquivos estão separados, será preciso juntá-los cruzando o país.

___

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
#Leitura dos arquivos em Excel
co2 = pd.read_excel('indicator CDIAC carbon_dioxide_emissions_per_capita.xlsx')
income = pd.read_excel('indicator gapminder gdp_per_capita_ppp.xlsx')

In [ ]:
#É possível verificar que cada linha representa um país e as colunas representam o ano
co2.head()

In [ ]:
#O mesmo se aplica a renda.
income.head()

___
### Inner Join

Vamos agora juntar as duas tabelas via país, selecionar apenas o ano de 2010 e remover os NaNs.

Ao final vamos fazer o gráfico de dispersão das duas variáveis.

**Sugestão**: pesquise sobre a função DataFrame.join(), pode ser muito útil no futuro.

In [ ]:
#Cruza as duas tabelas via país
df = co2.set_index('CO2 per capita').join(income.set_index('GDP per capita'), how='inner', lsuffix='_co2', rsuffix='_income')
#Seleciona o ano de 2010 e remove os NaNs
df = df[['2010_co2','2010_income']].dropna()
#Transforma a renda na escala de milhares de dólares
df['2010_income'] /= 1000

#Plota o gráfico de dispersão
df.plot.scatter('2010_income','2010_co2');

___
### Regressão

Vamos tentar agora ajustar um modelo sobre os dados. A primeira tentativa será ajustar um reta:

$$y_i=\beta_0+\beta_1x_i+\epsilon_i$$


Esse modelo é muito parecido com o visto na aula 06, com algumas mudanças:
1. Agora utilizaremos $\beta_i$ para se referir aos coeficientes.
1. Existe um termo $\epsilon_i$ para representar os resíduos.

O primeiro passo agora é calcular os valores da regressão. Existem diversas formas de estimar os $\beta$s, vamos utilizar o método de **Mínimos Quadrados Ordinários (MQO ou OLS - Ordinary Least Squares em inglês)**. 

$$\hat{\beta}_0=\bar{y}-\hat{\beta}_1\bar{x}$$

$$\hat{\beta}_1=\frac{S_{XY}}{S_{XX}}=\frac{\sum_{i=1}^n(x_i-\bar{x})(y_i-\bar{y})}{\sum_{i=1}^n(x_i-\bar{x})^2}$$

**Obs: Acompanhar e anotar o desenvolvimento no quadro.**
___
**Tarefa 1**: Calcule os valores de $\bar{x}$ e $\bar{y}$. Calcule também $S_{XX}$ e $S_{XY}$, em seguida calcule o $\hat{\beta}_1$ e o $\hat{\beta}_0$

___
Agora calcular o vetor de resíduos, dado por:
$$\epsilon_i=y_i-(\hat{\beta}_0+\hat{\beta}_1x_i)$$

**Tarefa 2**: Plote o histograma dos resíduos. Parece uma Normal?

___
Agora que já aprendemos Teste de Hipóteses, podemos verificar se os Betas são realmente relevantes. para tal vamos testar:

$$H_0: \beta_i=0$$
$$H_1: \beta_i\neq0$$

Onde os betas terão a distribuição:

$$\beta_0\sim N(0,\sigma_{\beta_0}^2)\sim N\left(0,\sigma^2\left(\frac{1}{n}+\frac{\bar{x}^2}{S_{XX}}\right)\right)$$
$$\beta_1\sim N(0,\sigma_{\beta_1}^2)\sim N\left(0,\frac{\sigma^2}{S_{XX}}\right)$$

Como não conhecemos o $\sigma^2$, vamos utilizar um valor estimado $\hat{\sigma}^2$:

$$\hat{\sigma}^2=\frac{SS_E}{n-2}=\frac{\sum_{i=1}^n(y_i-(\hat{\beta}_0+\hat{\beta}_1x_i))^2}{n-2}$$

$SS_E$ também é conhecido como Sum of Square Errors (ou SQE em português).
___
**Tarefa 3**: Calcular o erro padrão $\hat{\sigma}$. Cacular também os erros $\hat{\sigma}_{\beta_0}$ e $\hat{\sigma}_{\beta_1}$.


**Tarefa 4**: Calcule o **valor-p** para os betas e verifique se rejeitamos ou não $H_0$. Utilize $\alpha=5\%$.

**Tarefa 5:** Calcule o **intervalo de confiança** para os betas.

___
### Coeficiente de Determinação

$$R^2=1-\frac{SS_E}{SS_T}$$

Onde:

$$SS_T=SS_R+SS_E$$
$$SS_R=\sum_{i=1}^n(\hat{y}-\bar{y})^2$$

$SS_T$: Total Sum of Square

$SS_R$: Regression Sum of Square
___
**Tarefa 6:** Calcule o $R^2$. Interprete.

___
### Grand Finale

Agora que estão familiarizados com algumas medidas, rode a função OLS() da biblioteca statsmodel:

In [ ]:
import numpy as np
import statsmodels.api as sm

Y = df['2010_co2']
X = df['2010_income']
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.summary()

___
### Para ir além...

Referência: http://www.statsmodels.org/dev/diagnostic.html

**Tarefa 7**: Você consegue interpretar alguns dos resultados acima? Você acha que os resíduos formam uma normal? Fale um pouco sobre a homocedasticidade do modelo.

No caso de rejeição de homecedasticidade, é necessário usar uma modelagem que permita heterocedasticidade (Mínimos Quadrados Generalizados).
Veja, por exemplo, http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html